In [1]:
import os
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import pickle as pkl
import cv2
import numpy as np
from sklearn import model_selection
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization, Dense, Add, ReLU
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import MeanIoU
import random

## Check TF GPU

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## Load csv and paths

In [4]:
csv_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train.csv"
train_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train_images/"
train_masks_path = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train_masks/"

csv_reducido = r"C:\Users\Agustin\Documents\RepositorioCEIA_Git\VpC2_Datasets\severstal-steel-defect-detection/train_4sigmoid.csv"

In [5]:
n_w = 1600
n_h = 256

## Create a more balanced dataset with defects and extra 10% without defects

In [6]:
defective_csv = pd.read_csv(csv_reducido)
all_img = [value for value in listdir(train_path)]
defective_img = [value for value in listdir(train_path) if value in list(defective_csv['ImageId'])]
nondefective_img = [value for value in listdir(train_path) if value not in list(defective_csv['ImageId'])]

In [7]:
print(len(defective_img))
print(len(nondefective_img))

2748
9820


In [8]:
idx_aux= np.random.permutation(len(nondefective_img))[0: int(len(defective_img)*0.2)-1]
defective_img = defective_img + [nondefective_img[value] for value in idx_aux]

In [9]:
len(defective_img)

3296

## Train, Test and Validation 

In [10]:
x_train, x_test = model_selection.train_test_split( defective_img, test_size=0.1, random_state=5)
x_train, x_valid = model_selection.train_test_split( x_train, test_size=0.1, random_state=5)

## Functions for inline (RAM) Y defect mask creation

In [11]:
def decode_pixels(pixels_str):
    """
    This function receives a list which contains the pixels as a space-separated code.
    For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask
    """
    
    pixels_encoded_list = pixels_str.split(' ')
    pixels_list = []
    for i in range(0, len(pixels_encoded_list), 2):
        # Add to the pixels list all the pixels encoded by the 2 digits
        pixels_list += range(
            int(pixels_encoded_list[i]) - 1,
            int(pixels_encoded_list[i]) + int(pixels_encoded_list[i + 1]) - 1,
        )
        
    # Return the decoded list    
    return pixels_list

## Data Augmentation Functions

In [12]:
def flip(image, mask , vflip=False, hflip=False):
    '''
    Flip the image
    :param image: image to be processed
    :param vflip: whether to flip the image vertically
    :param hflip: whether to flip the image horizontally
    '''
    if hflip or vflip:
        if hflip and vflip:
            c = -1
        else:
            c = 0 if vflip else 1
        image = cv2.flip(image, flipCode=c)
        mask = cv2.flip(mask, flipCode=c)
    return image, mask

def rotate(image, mask, angle=90, scale=1.0):
    '''
    Rotate the image
    :param image: image to be processed
    :param angle: Rotation angle in degrees. Positive values mean counter-clockwise rotation (the coordinate origin is assumed to be the top-left corner).
    :param scale: Isotropic scale factor.
    '''
    w = image.shape[1]
    h = image.shape[0]
    #rotate matrix
    M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
    #rotate
    image = cv2.warpAffine(image,M,(w,h))
    mask = cv2.warpAffine(mask,M,(w,h))
    return image, mask
    
def shift(image, mask, px, py):
    w = image.shape[1]
    h = image.shape[0]
    #c = image.shape[2]
    
    img = np.zeros((h,w))
    m = np.zeros((h,w))   
    
    if (py > 0) and (px > 0):
        img[py:, px:] = image[:-py, :-px]
        m[py:, px:] = mask[:-py, :-px]
    elif (py < 0) and (px < 0):
        img[:py, :px] = image[-py:, -px:]
        m[:py, :px] = mask[-py:, -px:]
    elif (py > 0) and (px < 0):
        img[py:, :px] = image[:-py, -px:]
        m[py:, :px] = mask[:-py, -px:]
    elif (py < 0) and (px > 0):
        img[:py, px:] = image[-py:, :-px]
        m[:py, px:] = mask[-py:, :-px]
    
    return img, m

## Data generator for train

In [13]:
# Train Data Generator
def data_generator(ids, batch_size):
    Xs = []; ys = []
    
    while True:
        i = 0
        random.shuffle(ids)
        aug_shift = np.random.randint(2, size=len(ids))
        aug_rot = np.random.randint(2, size=len(ids))
        aug_flip = np.random.randint(2, size=len(ids))
        for id in ids:
            img = cv2.imread(train_path + id)
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img = img / 255.
            
            mask = make_mask(id)
            
            img=cv2.resize(img, (int(img.shape[1]/2),int(img.shape[0]/2)))
            mask=cv2.resize(mask, (int(mask.shape[1]/2),int(mask.shape[0]/2)))
            
            #if aug_shift[i] == 1:
            #    img, mask = shift(img, mask,
            #                      px=np.random.randint(low = -int(img.shape[1]/10),high = int(img.shape[1]/10)),
            #                      py=np.random.randint(low = -int(img.shape[0]/10),high = int(img.shape[0]/10))
            #                     )
            #if aug_rot[i] == 1:
            #    img, mask = rotate(img, mask, angle=np.random.randint(-10,10), scale=1.0 + (random.randint(0,20)/10) )
            #if aug_flip[i] == 1:
            #    img, mask = flip(img, mask, vflip=True, hflip=False)
            #mask = (mask > 0.2) * 1.0
            #
            #i+=1
            
            Xs.append(img); ys.append(mask)
            if len(Xs) == batch_size:
                X = np.array(Xs); y = np.array(ys)
                Xs = []; ys = []
                yield (X, y)

### Additional usefull metrics

In [14]:
smooth = 1
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels=4):
    dice=0
    for index in range(numLabels):
        dice -= dice_coef(y_true[:,:,index], y_pred[:,:,index])
    return dice

def dice_coef_nd(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return 2.*intersection, (np.sum(y_true_f)+np.sum(y_pred_f))

def dice_loss(y_true,y_pred):
    return (1 - dice_coef(y_true,y_pred))

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (1 - ((true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)))

# ResUnet - Inception

In [15]:
def inception_block(prevlayer, a, b):
    
    conva = Conv2D(a, (1, 1), activation='relu', padding='same')(prevlayer)
    conva = Conv2D(b, (3, 3), activation=None, padding='same')(conva)
    conva = BatchNormalization()(conva)
    conva = ReLU()(conva)
    
    
    convb = Conv2D(a, (1, 1), activation='relu', padding='same')(prevlayer)
    convb = Conv2D(b, (5, 5), activation=None, padding='same')(convb)
    convb = BatchNormalization()(convb)
    convb = ReLU()(convb)

    convc = Conv2D(b, (1, 1), activation=None, padding='same')(prevlayer)
    convc = BatchNormalization()(convc)
    convc = ReLU()(convc)
        
    convd = MaxPooling2D(pool_size=(3, 3), strides=(1,1), padding= 'same')(prevlayer)
    convd = Conv2D(b, (1, 1), activation=None, padding='same')(convd)
    convd = BatchNormalization()(convd)
    convd = ReLU()(convd)
        
        
    up = concatenate([conva, convb, convc, convd])
    return up

In [16]:
def residual_block (input_layer, neurons):
    
    conv1 = Conv2D(neurons , (5, 5), activation=None, padding="same",kernel_initializer='he_normal')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)
    conv1 = Conv2D(neurons, (5, 5), activation=None, padding="same",kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)
    conv1 = Conv2D(neurons, (5, 5), activation=None, padding="same",kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    
    conv1_skip = Conv2D(neurons, 1, 1, activation=None, padding="same",kernel_initializer='he_normal')(input_layer)
    conv1_skip = BatchNormalization()(conv1_skip)
    
    added1 = Add()([conv1_skip, conv1])
    relu1 = ReLU()(added1)
    
    return relu1

In [17]:
def build_model(input_layer, start_neurons):
    
    conv1 = residual_block(input_layer,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    #pool1 = Dropout(0.25)(pool1)

    conv2 = residual_block(pool1,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    #pool2 = Dropout(0.5)(pool2)

    conv3 = residual_block(pool2,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4)
    pool3 = MaxPooling2D((2, 2))(conv3)
    #pool3 = Dropout(0.5)(pool3)

    conv4 = residual_block(pool3,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8)
    pool4 = MaxPooling2D((2, 2))(conv4)
    #pool4 = Dropout(0.5)(pool4)

    # Mitad
    convm = inception_block (pool4, start_neurons * 4, start_neurons * 4)
    convm = inception_block (convm, start_neurons * 4, start_neurons * 4)
    convm = inception_block (convm, start_neurons * 4, start_neurons * 4)
    convm = inception_block (convm, start_neurons * 4, start_neurons * 4)

    deconv4 = Conv2DTranspose(start_neurons * 8, (5, 5), strides=(2, 2), padding="same",kernel_initializer='he_normal')(convm)
    uconv4 = concatenate([deconv4, conv4])
    #uconv4 = Dropout(0.5)(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8)

    deconv3 = Conv2DTranspose(start_neurons * 4, (5, 5), strides=(2, 2), padding="same",kernel_initializer='he_normal')(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    #uconv3 = Dropout(0.5)(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4)

    deconv2 = Conv2DTranspose(start_neurons * 2, (5, 5), strides=(2, 2), padding="same",kernel_initializer='he_normal')(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    #uconv2 = Dropout(0.5)(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (5, 5), strides=(2, 2), padding="same",kernel_initializer='he_normal')(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    #uconv1 = Dropout(0.5)(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid",kernel_initializer='he_normal')(uconv1)
    
    return output_layer

input_layer = Input((128, 800, 1))
output_layer = build_model(input_layer, 8)

model = Model(input_layer, output_layer)


In [18]:
model.load_weights('./weights/resUnetInceptionV5/resUnetInceptionV5')

## Model 2
## U-Net Model with 4 sigmoid to detect presence or not of each class independently

In [19]:
n_class = 4
def make_mask(ImageId):
    
    y_out=defective_csv[defective_csv['ImageId'].eq(ImageId)]
    
    image_pixels = np.zeros((n_h, n_w,n_class), dtype=np.float32)

    for index, row in y_out.iterrows():

        # Generate the mask of the error type
        mask = np.zeros((n_h * n_w, 1), dtype=np.float32)
        mask[decode_pixels(row['EncodedPixels'])] = 1
        mask = mask.reshape(n_w, n_h)
        mask = mask.T
    
        # Update channel of the given error
        image_pixels [:,:,int(row['ClassId'])-1] = np.logical_or(image_pixels [:,:,int(row['ClassId'])-1],mask)
        
    return image_pixels

In [20]:
def build_model2(input_layer, start_neurons):
    
    uconv1 = residual_block(input_layer,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1)
    output_layer1 = Conv2D(1, (1,1), padding="same", activation="sigmoid",kernel_initializer='he_normal')(uconv1)
    
    uconv2 = residual_block(input_layer,start_neurons * 1)
    uconv2 = residual_block(uconv2,start_neurons * 1)
    output_layer2 = Conv2D(1, (1,1), padding="same", activation="sigmoid",kernel_initializer='he_normal')(uconv2)
    
    uconv3 = residual_block(input_layer,start_neurons * 1)
    uconv3 = residual_block(uconv3,start_neurons * 1)
    output_layer3 = Conv2D(1, (1,1), padding="same", activation="sigmoid",kernel_initializer='he_normal')(uconv3)
    
    uconv4 = residual_block(input_layer,start_neurons * 1)
    uconv4 = residual_block(uconv4,start_neurons * 1)
    output_layer4 = Conv2D(1, (1,1), padding="same", activation="sigmoid",kernel_initializer='he_normal')(uconv4)
    
    uconvf = concatenate([output_layer1, output_layer2, output_layer3, output_layer4])
    
    
    return uconvf

output_layer = build_model2(model.layers[-4].output,4)

model2 = Model(model.layers[0].input, output_layer)

In [22]:
model2.load_weights('./weights/resUnetInception4Sigmoid_5epocas_v1/resUnetInception4Sigmoid_25epocas_v2')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000284F25DC9D0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002839F8ADA60>).


ValueError: Shapes (5, 5, 8, 8) and (1, 1, 8, 8) are incompatible

In [21]:
optimizer = Adam()
#model2.compile(loss=tversky, optimizer=optimizer, metrics=[dice_coef])
#model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[dice_coef])
model2.compile(optimizer, dice_loss, metrics=[dice_coef])

In [22]:
model2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 800, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 800, 8)  208         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 800, 8)  32          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 128, 800, 8)  0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
epochs = 25
batch_size = 16

train_generator = data_generator(x_train, batch_size)
validation_generator = data_generator(x_valid, batch_size)

train_steps_per_epoch=int(len(x_train)/batch_size)
validation_steps_per_epoch=int(len(x_valid)/batch_size)

history = model2.fit_generator(generator=train_generator,
                               steps_per_epoch=train_steps_per_epoch,
                               epochs=epochs,
                               validation_data = validation_generator,
                               validation_steps = validation_steps_per_epoch
                               #callbacks=[early_stopping, model_checkpoint, reduce_lr])
                              )

Epoch 1/25
 25/166 [===>..........................] - ETA: 2:10 - loss: 0.4074 - dice_coef: 0.5926

In [ ]:
model2.save_weights('./weights/resUnetInception4Sigmoid_5epocas_v1/resUnetInception4Sigmoid_25epocas_v3')

In [ ]:
test_generator = data_generator(x_test,batch_size=1)

for i in range(10):
    data_test = next(test_generator)
    prediction=model2.predict(data_test[0])
    plt.imshow(data_test[0][0],cmap='gray', vmin=0, vmax=1)
    plt.show()
    for j in range(data_test[1][0].shape[2]):
        plt.imshow(data_test[1][0][:,:,j]*128)
        plt.show()
        plt.imshow(prediction[0][:,:,j]*128)
        plt.show()


In [ ]:
test_generator = data_generator(x_test,batch_size=1)

tpn=0
total = 0
intersection = 0
union = 0
acum_sum=0

for i in range(len(x_test)):
    
    data_test = next(test_generator)
    y_true = data_test[1]
    y_pred =  (model2.predict(data_test[0])>=0.5)
    
    #Pixel Accuracy
    tpn += np.sum(y_true==y_pred) / y_true.shape[3]
    total += y_true.shape[1]*y_true.shape[2]
    
    intersection += np.sum(y_true * y_pred,axis=(1,2))
    union += np.sum(np.logical_or(y_true,y_pred), axis=(1,2))
    
    acum_sum += np.sum(y_true, axis=(1,2)) + np.sum(y_pred, axis=(1,2))
    
pixel_accuracy = tpn/total

union[union==0]=1
mean_iou = (intersection / union)
    
acum_sum[acum_sum==0]=1
dice = (2* intersection / acum_sum)

In [ ]:
test_generator = data_generator(all_img,batch_size=1)

tpn=0
total = 0
intersection = 0
union = 0
acum_sum=0

for i in range(len(all_img)):
    
    data_test = next(test_generator)
    y_true = data_test[1]
    y_pred =  (model2.predict(data_test[0])>=0.5)
    
    #Pixel Accuracy
    tpn += np.sum(y_true==y_pred) / y_true.shape[3]
    total += y_true.shape[1]*y_true.shape[2]
    
    intersection += np.sum(y_true * y_pred,axis=(1,2))
    union += np.sum(np.logical_or(y_true,y_pred), axis=(1,2))
    
    acum_sum += np.sum(y_true, axis=(1,2)) + np.sum(y_pred, axis=(1,2))
    
pixel_accuracy = tpn/total

union[union==0]=1
mean_iou = (intersection / union)
    
acum_sum[acum_sum==0]=1
dice = (2* intersection / acum_sum)

In [ ]:
union

In [ ]:
pixel_accuracy

In [ ]:
mean_iou

In [ ]:
dice